In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install openai

In [ ]:
!pip install "unstructured[all-docs]"

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
OPENAI_API_KEY = 'openai_api_key'
loader = UnstructuredFileLoader('/Users/sathyaselvam/Downloads/revengeofthesith.pdf/', mode="elements")

In [ ]:
data = loader.load()

In [ ]:
print(f'You have {len(data)} document(s) in your data')
print(f'There are {len(data[0].page_content)} characters in your data')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 7000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)

In [ ]:
print(f'You have {len(texts)} document(s) in your data')
print(texts)

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)

In [ ]:
PINECONE_API_KEY = 'pinecone_api_key'
pinecone.init(
    api_key = PINECONE_API_KEY,
    environment = 'northamerica-northeast1-gcp'
)
index_name = 'index1'

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name = index_name)

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
llm = OpenAI(temperature = 0.9, openai_api_key = OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type='stuff', verbose = True)

In [ ]:
query = "Who is Obi-wan Kenobi"
docs = docsearch.similarity_search(query)

In [ ]:
chain.run(input_documents=docs, question=query)